# 10-3 实现混淆矩阵、精准率和召回率

+ 混淆矩阵、精准率、召回率根据定义能够很轻松地实现
+ `sklearn.metrics` 模块中提供了 `confusion_matrix`、`precision_score`、`recall_score` 实现，可以直接使用。

$$
{\rm 精准率} = \frac{TP}{TP + FP}
$$


$$
{\rm 召回率} = \frac{TP}{TP + FN}
$$

## 预测手写数字 digits

In [1]:
import numpy as np
from sklearn import datasets

digists = datasets.load_digits()
X = digists.data
# 深层拷贝，因为我们要修改 y 的值来做实验
y = digists.target.copy()

y[digists.target == 9] = 1
y[digists.target < 9] = 0

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [3]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.9755555555555555

如何解释这个 0.97555555555555551 ？

In [4]:
y_log_predict = log_reg.predict(X_test)
y_log_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [5]:
sum(y_log_predict)

38

In [6]:
sum(y_test)

45

In [7]:
len(y_test)

450

In [8]:
sum(y_log_predict == y_test) / len(y_test)

0.9755555555555555

In [9]:
def TN(y_true, y_predict):
    '''预测的是 0 ，但是真实值是 0 的数量'''
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 0) & (y_predict == 0))


TN(y_test, y_log_predict)

403

___

## 位运算符

Python 运算符 | 菜鸟教程  http://www.runoob.com/python/python-operators.html#ysf5  
+ `&`	按位与运算符：参与运算的两个值，如果两个相应位都为 1 ，则该位的结果为 1 ，否则为 0 。
+ `|`	按位或运算符：只要对应的二个二进位有一个为 1 时，结果位就为 1 。
+ `^`	按位异或运算符：当两对应的**二进位**相异时，结果为 1。
+ `~`	按位取反运算符：对数据的**每个二进制位**取反，即把 1 变为 0 ，把 0 变为 1 。**~x 类似于 -x-1**。

In [10]:
a1 = np.array([1, 1, 1])
b1 = np.array([0, 0, 0])

a1 & b1

array([0, 0, 0])

In [11]:
a1 | b1

array([1, 1, 1])

In [12]:
~a1

array([-2, -2, -2])

---

In [13]:
def FP(y_true, y_predict):
    '''预测的是 1 ，但是真实值是 0 的数量'''
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 0) & (y_predict == 1))


FP(y_test, y_log_predict)

2

In [14]:
def FN(y_true, y_predict):
    '''预测的是 0 ，但是真实值是 1 的数量'''
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 1) & (y_predict == 0))


FN(y_test, y_log_predict)

9

In [15]:
def TP(y_true, y_predict):
    '''预测的是 1 ，但是真实值是 1 的数量'''
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 1) & (y_predict == 1))


TP(y_test, y_log_predict)

36

计算混淆矩阵


|       | 预测值：0 | 预测值：1 |
| :---: | :---: | :---: |
| 真实值：0 |  TN   |  FP   |
| 真实值：1 |  FN   |  TP   |

In [16]:
def confusion_matrix(y_true, y_predict):
    return np.array([
        [TN(y_true, y_predict), FP(y_true, y_predict)],
        [FN(y_true, y_predict), TP(y_true, y_predict)]
    ])


confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

### 计算精准率和召回率

$$
{\rm 精准率} = \frac{TP}{TP + FP}
$$

In [17]:
def precision_score(y_true, y_predict):
    tp = TP(y_test, y_log_predict)
    fp = FP(y_test, y_log_predict)
    try:
        return tp / (tp + fp)
    except:
        return 0.0


precision_score(y_test, y_log_predict)

0.9473684210526315

$$
{\rm 召回率} = \frac{TP}{TP + FN}
$$

In [18]:
def recall_score(y_true, y_predict):
    tp = TP(y_test, y_log_predict)
    fn = FN(y_test, y_log_predict)
    try:
        return tp / (tp + fn)
    except:
        return 0.0


recall_score(y_test, y_log_predict)

0.8

## scikit-learn中的混淆矩阵，精准率和召回率

In [19]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [20]:
from sklearn.metrics import precision_score

precision_score(y_test, y_log_predict)

0.9473684210526315

In [21]:
from sklearn.metrics import recall_score

recall_score(y_test, y_log_predict)

0.8